In [2]:
#model = pickle.load(open("lgbm_v_18_fold_all.bin",'rb')) 


In [3]:
# model.num_trees()
# model.feature_name()
# model.params

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import random
import gc
import lightgbm as lgb
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import feather
import os
import glob
import csv   
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import datetime

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Using TensorFlow backend.
/Users/pgut682g/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/pgut682g/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/pgut682g/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/p

Set seed

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Functions

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Load data

In [5]:
path = '/Users/pgut682g/Projects/Projects/ashare'

In [6]:
%%time
train = feather.read_dataframe(path + '/data-vol-3/train_filled_na.feather')

CPU times: user 13.9 s, sys: 39.6 s, total: 53.5 s
Wall time: 1min 53s


In [7]:
%%time
test_for_train = feather.read_dataframe(path + '/data-vol-3/test_filled_all_na.feather')

CPU times: user 25.7 s, sys: 1min 18s, total: 1min 44s
Wall time: 4min 33s


# Train train data only

In [8]:
train = reduce_mem_usage(train)

Mem. usage decreased to 2718.42 Mb (64.9% reduction)


In [9]:
train['hour'] = train['utc_time'].dt.hour

In [11]:
train["primary_use"].unique()

[Education, Lodging/residential, Office, Entertainment/public assembly, Other, ..., Healthcare, Utility, Technology/science, Manufacturing/industrial, Services]
Length: 16
Categories (16, object): [Education, Lodging/residential, Office, Entertainment/public assembly, ..., Utility, Technology/science, Manufacturing/industrial, Services]

In [12]:
test_for_train["primary_use"].unique()

array([ 0,  4,  6,  1,  7, 11,  8,  9, 15,  2, 10,  3, 14, 13,  5, 12])

In [26]:
le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])

In [ ]:
train["primary_use"].unique()

In [13]:
categoricals = ['meter',
 'primary_use',
 'building_id',
 'site_id',              
]

numericals = ['square_feet',
 'floor_count',
 'new_floor_count',
 'is_holiday',
 'hour',
 'weekday_cos',
 'weekday_sin',
 'weekday',
 'hour_cos',
 'hour_sin',
 'year_cos',
 'year_sin',
 'is_day_saving',
 'air_temperature',
 'age',
 'dew_temperature',
 'sea_level_pressure',
 'new_sea_level_pressure',
 'new_dew_temperature',
 'new_wind_speed',
 'new_air_temperature',
 'humidity',
 'latitude',
 'irradiance',
 'feels_like',
 'new_feels_like',
 'radiation',
 'air_temperature_mean_lag72',
 'air_temperature_max_lag72',
 'air_temperature_min_lag72',
 'air_temperature_std_lag72',
 'cloudCover_mean_lag72',
 'cloudCover_std_lag72',
 'new_wind_direction_cos',
 'new_wind_direction_sin']

feat_cols = categoricals + numericals

In [14]:
train['year_built'].fillna(-999, inplace=True)
train['new_year_built'].fillna(-999, inplace=True)
train['age'] = train['utc_time'].dt.year - train['year_built']

In [15]:
train = train.dropna() 

In [16]:
target = np.log1p(train["meter_reading"])

del train["meter_reading"] 
#train = train.drop(drop_cols, axis = 1)

In [17]:
train = train.rename(columns={"utc_time":"timestamp"})

In [ ]:
#train = train[(train['suspicious_1'] != 1) & (train['suspicious_2'] != 1)]

In [18]:
def model(dense_dim_1=128, dense_dim_2=64, dense_dim_3=32, dense_dim_4=16, 
dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=3e-4): 

    #Inputs
    site_id = Input(shape=[1], name="site_id")
    building_id = Input(shape=[1], name="building_id")
    meter = Input(shape=[1], name="meter")
    primary_use = Input(shape=[1], name="primary_use")
    is_holiday = Input(shape=[1], name="is_holiday")
    is_day_saving = Input(shape=[1], name="is_day_saving")
    
    hour_cos = Input(shape=[1], name="hour_cos")
    hour = Input(shape=[1], name="hour")
    air_temperature = Input(shape=[1], name="air_temperature")
    dew_temperature = Input(shape=[1], name="dew_temperature")  
    floor_count = Input(shape=[1], name="floor_count")
    new_floor_count = Input(shape=[1], name="new_floor_count")
    square_feet = Input(shape=[1], name="square_feet")
    irradiance = Input(shape=[1], name="irradiance")
    air_temperature_mean_lag72 = Input(shape=[1], name="air_temperature_mean_lag72")
    
    weekday_cos = Input(shape=[1], name="weekday_cos")
    weekday_sin = Input(shape=[1], name="weekday_sin")
    weekday = Input(shape=[1], name="weekday")
    hour_sin = Input(shape=[1], name="hour_sin")
    year_cos = Input(shape=[1], name="year_cos")
    year_sin = Input(shape=[1], name="year_sin")

    age = Input(shape=[1], name="age")
    sea_level_pressure = Input(shape=[1], name="sea_level_pressure")
    new_sea_level_pressure = Input(shape=[1], name="new_sea_level_pressure")
    new_dew_temperature = Input(shape=[1], name="new_dew_temperature")
    new_wind_speed = Input(shape=[1], name="new_wind_speed")
    new_air_temperature = Input(shape=[1], name="new_air_temperature")
    humidity = Input(shape=[1], name="humidity")
    latitude = Input(shape=[1], name="latitude")
    feels_like = Input(shape=[1], name="feels_like")
    new_feels_like = Input(shape=[1], name="new_feels_like")
    radiation = Input(shape=[1], name="radiation")
    air_temperature_max_lag72 = Input(shape=[1], name="air_temperature_max_lag72")
    air_temperature_min_lag72 = Input(shape=[1], name="air_temperature_min_lag72")
    air_temperature_std_lag72 = Input(shape=[1], name="air_temperature_std_lag72")
    air_temperature_mean_lag72 = Input(shape=[1], name="air_temperature_mean_lag72") 
    cloudCover_mean_lag72 = Input(shape=[1], name="cloudCover_mean_lag72")
    cloudCover_std_lag72 = Input(shape=[1], name="cloudCover_std_lag72")
    new_wind_direction_cos = Input(shape=[1], name="new_wind_direction_cos")
    new_wind_direction_sin = Input(shape=[1], name="new_wind_direction_sin")
   
    #Embeddings layers
    emb_site_id = Embedding(16, 8)(site_id)
    emb_building_id = Embedding(1449, 50)(building_id)
    emb_meter = Embedding(4, 2)(meter)
    emb_primary_use = Embedding(16, 8)(primary_use)
#     emb_hour_cos = Embedding(24, 12)(hour_cos)
#     emb_hour_sin = Embedding(24, 12)(hour_sin)
    emb_hour = Embedding(24, 12)(hour)
#     emb_weekday_cos = Embedding(7, 3)(weekday_cos) 
#     emb_weekday_sin = Embedding(7, 3)(weekday_sin) 
    emb_weekday = Embedding(7, 3)(weekday)

    concat_emb = concatenate([
           Flatten() (emb_site_id)
         , Flatten() (emb_building_id)
         , Flatten() (emb_meter)
         , Flatten() (emb_primary_use)
         , Flatten() (emb_hour) 
#          , Flatten() (emb_hour_sin)
         , Flatten() (emb_weekday)
#          , Flatten() (emb_weekday_cos)
#          , Flatten() (emb_weekday_sin)
    ])

    
    categ = Dropout(dropout1)(Dense(dense_dim_1,activation='relu') (concat_emb))
    categ = BatchNormalization()(categ)
    categ = Dropout(dropout2)(Dense(dense_dim_2,activation='relu') (categ))
    
    #main layer
    main_l = concatenate([
          categ,
          square_feet, floor_count,
             new_floor_count, is_holiday, is_day_saving,
            year_cos, year_sin, hour_cos, hour_sin, weekday_cos, weekday_sin, air_temperature,
            age, dew_temperature, sea_level_pressure, new_sea_level_pressure, new_dew_temperature,
             new_wind_speed, new_air_temperature, humidity, latitude, irradiance, feels_like,
            new_feels_like, radiation, air_temperature_mean_lag72,
             air_temperature_max_lag72, air_temperature_min_lag72,
            air_temperature_std_lag72, cloudCover_mean_lag72,
            cloudCover_std_lag72,new_wind_direction_cos,
             new_wind_direction_sin
    ])
      
    main_l = Dropout(dropout3)(Dense(dense_dim_3,activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    main_l = Dropout(dropout4)(Dense(dense_dim_4,activation='relu') (main_l))
    
    #output
    output = Dense(1) (main_l)

    model = Model([building_id, meter, site_id, primary_use, square_feet, floor_count,
             new_floor_count, is_holiday, weekday_cos, weekday_sin,
            weekday, hour_cos, hour_sin, hour, year_cos, year_sin, is_day_saving, air_temperature,
            age, dew_temperature, sea_level_pressure, new_sea_level_pressure, new_dew_temperature,
             new_wind_speed, new_air_temperature, humidity, latitude, irradiance, feels_like,
            new_feels_like, radiation, air_temperature_mean_lag72,
             air_temperature_max_lag72, air_temperature_min_lag72,
            air_temperature_std_lag72, cloudCover_mean_lag72,
            cloudCover_std_lag72,new_wind_direction_cos,
             new_wind_direction_sin], output)

    model.compile(optimizer = Adam(lr=lr),
                  loss= mse_loss,
                  metrics=[root_mean_squared_error])
    return model

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [19]:
def get_keras_data(df, num_cols, cat_cols):
    cols = num_cols + cat_cols
    X = {col: np.array(df[col]) for col in cols}
    return X

In [20]:
def train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold, patience=6):
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    model_checkpoint = ModelCheckpoint("model_" + str(fold) + ".hdf5",
                                       save_best_only=True, verbose=1, monitor='val_root_mean_squared_error', mode='min')

    hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=1,
                            callbacks=[early_stopping, model_checkpoint])

    keras_model = load_model("model_" + str(fold) + ".hdf5", custom_objects={'root_mean_squared_error': root_mean_squared_error})
    
    return keras_model

In [21]:
from sklearn.model_selection import KFold, StratifiedKFold

oof = np.zeros(len(train))
batch_size = 1024   
epochs = 100
models = []

folds = 3   
seed = 666

group = train['building_id'].astype(str) + "_" + (train['timestamp'].dt.month).astype(str)
groupSplit = GroupShuffleSplit(folds, test_size=0.2, random_state=7)
#folds = groupSplit.split(train[['radiation']], train['meter'], groups=group)

# kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

# for fold_n, (train_index, valid_index) in enumerate(kf.split(train, train['building_id'])):
for fold_n, (train_index, valid_index) in enumerate(groupSplit.split(train, target, groups=group)):
#for train_index, valid_index in groupSplit.split(train, target, groups=group):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_t = get_keras_data(X_train, numericals, categoricals)
    X_v = get_keras_data(X_valid, numericals, categoricals)
    
    keras_model = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16,  
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=3e-4)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=10)
    models.append(mod) 
    print('*'* 50)

Fold: 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2801774 samples, validate on 703477 samples
Epoch 1/100
2801774/2801774 [==============================] - 156s 56us/step - loss: 3.5164 - root_mean_squared_error: 1.7355 - val_loss: 1.4942 - val_root_mean_squared_error: 0.7888

Epoch 00001: val_root_mean_squared_error improved from inf to 0.78876, saving model to model_0.hdf5
Epoch 2/100
2801774/2801774 [==============================] - 145s 52us/step - loss: 1.5836 - root_mean_squared_error: 1.2570 - val_loss: 1.4542 - val_root_mean_squared_error: 0.7834

Epoch 00002: val_root_mean_squared_error improved from 0.78876 to 0.78340, saving model to model_0.hdf5
Epoch 3/100
2801774/2801774 [==============================] - 137s 49us/step - loss: 1.4196 - root_mean_squared_error: 1.1901 - v

KeyboardInterrupt: 

1042 folds 5
- Model 0: 0.69026, 0.9391
- Model 1: 0.57073, 0.8925
- Model 2: 0.66545, 0.8997
- Model 3: 0.59834, 0.9236
- Model 4: 0.53854, 0.9353

1024 folds 3
- model 0: 0.6287, 0.8934
- model 1: 0.57151, 0.8645
- model 2: 0.65070, 0.8678

1024 inf folds 3
- model 0: 0.56257, 0.8701
- model 1: 0.62672, 0.9321
- model 2: 0.57557, 0.9443

In [22]:
gc.collect()

163

In [28]:
test_for_train["primary_use"].unique()

array([ 0,  4,  6,  1,  7, 11,  8,  9, 15,  2, 10,  3, 14, 13,  5, 12])

In [24]:

test_for_train["timestamp"] = test_for_train["utc_time"]
test_for_train["age"] = test_for_train["utc_time"].dt.year - test_for_train["year_built"]

test_for_train["month"] = test_for_train["utc_time"].dt.month
test_for_train["day"] = test_for_train["utc_time"].dt.day

test_for_train["year_built"] = test_for_train.apply(
                                lambda row: row['new_year_built'] if np.isnan(row['year_built']) else row['new_year_built'],
                                axis=1
                                )

test_for_train["floor_count"] = test_for_train.apply(
                                lambda row: row['new_floor_count'] if np.isnan(row['floor_count']) else row['new_floor_count'],
                                axis=1
                                )

test_for_train["hour"] = test_for_train["utc_time"].dt.hour

/Users/pgut682g/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:564: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [ ]:
test = test_for_train[feat_cols]

In [ ]:
test['age'].replace([np.inf, -np.inf], -999, inplace=True)

In [ ]:
test_for_train.isna().sum()

In [ ]:
test_for_train = test_for_train.set_index(['site_id','day','month'])

air_temperature_filler = test_for_train.groupby(['site_id','day','month'])['air_temperature'].mean()
air_temperature_filler = pd.DataFrame(air_temperature_filler.fillna(method='ffill'),columns=['air_temperature'])
test_for_train.update(air_temperature_filler,overwrite=False)

cloud_coverage_filler = test_for_train.groupby(['site_id','day','month'])['cloud_coverage'].mean()
cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=['cloud_coverage'])
test_for_train.update(cloud_coverage_filler,overwrite=False)

dew_temperature_filler = test_for_train.groupby(['site_id','day','month'])['dew_temperature'].mean()
dew_temperature_filler = pd.DataFrame(dew_temperature_filler.fillna(method='ffill'),columns=['dew_temperature'])
test_for_train.update(dew_temperature_filler,overwrite=False)

precip_depth_1_hr_filler = test_for_train.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
precip_depth_1_hr_filler = pd.DataFrame(precip_depth_1_hr_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])
test_for_train.update(precip_depth_1_hr_filler,overwrite=False)

sea_level_pressure_filler = test_for_train.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
sea_level_pressure_filler = pd.DataFrame(sea_level_pressure_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
test_for_train.update(sea_level_pressure_filler,overwrite=False)

wind_speed_filler = test_for_train.groupby(['site_id','day','month'])['wind_speed'].mean()
wind_speed_filler = pd.DataFrame(wind_speed_filler.fillna(method='ffill'),columns=['wind_speed'])
test_for_train.update(wind_speed_filler,overwrite=False)

new_sea_level_pressure_filler = test_for_train.groupby(['site_id','day','month'])['new_sea_level_pressure'].mean()
new_sea_level_pressure_filler = pd.DataFrame(new_sea_level_pressure_filler.fillna(method='ffill'),columns=['new_sea_level_pressure'])
test_for_train.update(new_sea_level_pressure_filler,overwrite=False)

new_dew_temperature_filler = test_for_train.groupby(['site_id','day','month'])['new_dew_temperature'].mean()
new_dew_temperature_filler = pd.DataFrame(new_dew_temperature_filler.fillna(method='ffill'),columns=['new_dew_temperature'])
test_for_train.update(new_dew_temperature_filler,overwrite=False)

new_wind_speed_filler = test_for_train.groupby(['site_id','day','month'])['new_wind_speed'].mean()
new_wind_speed_filler = pd.DataFrame(new_wind_speed_filler.fillna(method='ffill'),columns=['new_wind_speed'])
test_for_train.update(new_wind_speed_filler,overwrite=False)

new_cloud_coverage_filler = test_for_train.groupby(['site_id','day','month'])['new_cloud_coverage'].mean()
new_cloud_coverage_filler = pd.DataFrame(new_cloud_coverage_filler.fillna(method='ffill'),columns=['new_cloud_coverage'])
test_for_train.update(new_cloud_coverage_filler,overwrite=False)

new_precip_depth_1_hr_filler = test_for_train.groupby(['site_id','day','month'])['new_precip_depth_1_hr'].mean()
new_precip_depth_1_hr_filler = pd.DataFrame(new_precip_depth_1_hr_filler.fillna(method='ffill'),columns=['new_precip_depth_1_hr'])
test_for_train.update(new_precip_depth_1_hr_filler,overwrite=False)

cloudCover_max_lag72_filler = test_for_train.groupby(['site_id','day','month'])['cloudCover_max_lag72'].mean()
cloudCover_max_lag72_filler = pd.DataFrame(cloudCover_max_lag72_filler.fillna(method='ffill'),columns=['cloudCover_max_lag72'])
test_for_train.update(cloudCover_max_lag72_filler,overwrite=False)

cloudCover_min_lag72_filler = test_for_train.groupby(['site_id','day','month'])['cloudCover_min_lag72'].mean()
cloudCover_min_lag72_filler = pd.DataFrame(cloudCover_min_lag72_filler.fillna(method='ffill'),columns=['cloudCover_min_lag72'])
test_for_train.update(cloudCover_min_lag72_filler,overwrite=False)

cloudCover_std_lag72_filler = test_for_train.groupby(['site_id','day','month'])['cloudCover_std_lag72'].mean()
cloudCover_std_lag72_filler = pd.DataFrame(cloudCover_std_lag72_filler.fillna(method='ffill'),columns=['cloudCover_std_lag72'])
test_for_train.update(cloudCover_std_lag72_filler,overwrite=False)

beaufort_scale_filler = test_for_train.groupby(['site_id','day','month'])['beaufort_scale'].mean()
beaufort_scale_filler = pd.DataFrame(beaufort_scale_filler.fillna(method='ffill'),columns=['beaufort_scale'])
test_for_train.update(beaufort_scale_filler,overwrite=False)

wind_direction_cos_filler = test_for_train.groupby(['site_id','day','month'])['wind_direction_cos'].mean()
wind_direction_cos_filler = pd.DataFrame(wind_direction_cos_filler.fillna(method='ffill'),columns=['wind_direction_cos'])
test_for_train.update(wind_direction_cos_filler,overwrite=False)

wind_direction_sin_filler = test_for_train.groupby(['site_id','day','month'])['wind_direction_sin'].mean()
wind_direction_sin_filler = pd.DataFrame(wind_direction_sin_filler.fillna(method='ffill'),columns=['wind_direction_sin'])
test_for_train.update(wind_direction_sin_filler,overwrite=False)

new_wind_direction_cos_filler = test_for_train.groupby(['site_id','day','month'])['new_wind_direction_cos'].mean()
new_wind_direction_cos_filler = pd.DataFrame(new_wind_direction_cos_filler.fillna(method='ffill'),columns=['new_wind_direction_cos'])
test_for_train.update(new_wind_direction_cos_filler,overwrite=False)

new_wind_direction_sin_filler = test_for_train.groupby(['site_id','day','month'])['new_wind_direction_sin'].mean()
new_wind_direction_sin_filler = pd.DataFrame(new_wind_direction_sin_filler.fillna(method='ffill'),columns=['new_wind_direction_sin'])
test_for_train.update(new_wind_direction_sin_filler,overwrite=False)

new_feels_like_filler = test_for_train.groupby(['site_id','day','month'])['new_feels_like'].mean()
new_feels_like_filler = pd.DataFrame(new_feels_like_filler.fillna(method='ffill'),columns=['new_feels_like'])
test_for_train.update(new_feels_like_filler,overwrite=False)

new_air_temperature_filler = test_for_train.groupby(['site_id','day','month'])['new_air_temperature'].mean()
new_air_temperature_filler = pd.DataFrame(new_air_temperature_filler.fillna(method='ffill'),columns=['new_air_temperature'])
test_for_train.update(new_air_temperature_filler,overwrite=False)

uvIndex_filler = test_for_train.groupby(['site_id','day','month'])['uvIndex'].mean()
uvIndex_filler = pd.DataFrame(uvIndex_filler.fillna(method='ffill'),columns=['uvIndex'])
test_for_train.update(uvIndex_filler,overwrite=False)

cloudCover_filler = test_for_train.groupby(['site_id','day','month'])['cloudCover'].mean()
cloudCover_filler = pd.DataFrame(cloudCover_filler.fillna(method='ffill'),columns=['cloudCover'])
test_for_train.update(cloudCover_filler,overwrite=False)

humidity_filler = test_for_train.groupby(['site_id','day','month'])['humidity'].mean()
humidity_filler = pd.DataFrame(humidity_filler.fillna(method='ffill'),columns=['humidity'])
test_for_train.update(humidity_filler,overwrite=False)

latitude_filler = test_for_train.groupby(['site_id','day','month'])['latitude'].mean()
latitude_filler = pd.DataFrame(latitude_filler.fillna(method='ffill'),columns=['latitude'])
test_for_train.update(latitude_filler,overwrite=False)

longitude_filler = test_for_train.groupby(['site_id','day','month'])['longitude'].mean()
longitude_filler = pd.DataFrame(longitude_filler.fillna(method='ffill'),columns=['longitude'])
test_for_train.update(longitude_filler,overwrite=False)

irradiance_filler = test_for_train.groupby(['site_id','day','month'])['irradiance'].mean()
irradiance_filler = pd.DataFrame(irradiance_filler.fillna(method='ffill'),columns=['irradiance'])
test_for_train.update(irradiance_filler,overwrite=False)

irradiance_cloud_filler = test_for_train.groupby(['site_id','day','month'])['irradiance_cloud'].mean()
irradiance_cloud_filler = pd.DataFrame(irradiance_cloud_filler.fillna(method='ffill'),columns=['irradiance_cloud'])
test_for_train.update(irradiance_cloud_filler,overwrite=False)

feels_like_filler = test_for_train.groupby(['site_id','day','month'])['feels_like'].mean()
feels_like_filler = pd.DataFrame(feels_like_filler.fillna(method='ffill'),columns=['feels_like'])
test_for_train.update(feels_like_filler,overwrite=False)

altitude_filler = test_for_train.groupby(['site_id','day','month'])['altitude'].mean()
altitude_filler = pd.DataFrame(altitude_filler.fillna(method='ffill'),columns=['altitude'])
test_for_train.update(altitude_filler,overwrite=False)

azimuth_filler = test_for_train.groupby(['site_id','day','month'])['azimuth'].mean()
azimuth_filler = pd.DataFrame(azimuth_filler.fillna(method='ffill'),columns=['azimuth'])
test_for_train.update(azimuth_filler,overwrite=False)

radiation_filler = test_for_train.groupby(['site_id','day','month'])['radiation'].mean()
radiation_filler = pd.DataFrame(radiation_filler.fillna(method='ffill'),columns=['radiation'])
test_for_train.update(radiation_filler,overwrite=False)

air_temperature_mean_lag72_filler = test_for_train.groupby(['site_id','day','month'])['air_temperature_mean_lag72'].mean()
air_temperature_mean_lag72_filler = pd.DataFrame(air_temperature_mean_lag72_filler.fillna(method='ffill'),columns=['air_temperature_mean_lag72'])
test_for_train.update(air_temperature_mean_lag72_filler,overwrite=False)

air_temperature_max_lag72_filler = test_for_train.groupby(['site_id','day','month'])['air_temperature_max_lag72'].mean()
air_temperature_max_lag72_filler = pd.DataFrame(air_temperature_max_lag72_filler.fillna(method='ffill'),columns=['air_temperature_max_lag72'])
test_for_train.update(air_temperature_max_lag72_filler,overwrite=False)

air_temperature_min_lag72_filler = test_for_train.groupby(['site_id','day','month'])['air_temperature_min_lag72'].mean()
air_temperature_min_lag72_filler = pd.DataFrame(air_temperature_min_lag72_filler.fillna(method='ffill'),columns=['air_temperature_min_lag72'])
test_for_train.update(air_temperature_min_lag72_filler,overwrite=False)

air_temperature_std_lag72_filler = test_for_train.groupby(['site_id','day','month'])['air_temperature_std_lag72'].mean()
air_temperature_std_lag72_filler = pd.DataFrame(air_temperature_std_lag72_filler.fillna(method='ffill'),columns=['air_temperature_std_lag72'])
test_for_train.update(air_temperature_std_lag72_filler,overwrite=False)

cloudCover_mean_lag72_filler = test_for_train.groupby(['site_id','day','month'])['cloudCover_mean_lag72'].mean()
cloudCover_mean_lag72_filler = pd.DataFrame(cloudCover_mean_lag72_filler.fillna(method='ffill'),columns=['cloudCover_mean_lag72'])
test_for_train.update(cloudCover_mean_lag72_filler,overwrite=False)


In [ ]:
test_for_train = test_for_train.reset_index()

In [ ]:
test = reduce_mem_usage(test)

In [ ]:
from keras.models import load_model

models = []
#mod = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=3e-4)

mod = load_model('model_building_id_month_1024_3_fold/model_0.hdf5',custom_objects={"root_mean_squared_error": root_mean_squared_error})
models.append(mod) 
mod = load_model('model_building_id_month_1024_3_fold/model_1.hdf5',custom_objects={"root_mean_squared_error": root_mean_squared_error})
models.append(mod)
mod = load_model('model_building_id_month_1024_3_fold/model_2.hdf5',custom_objects={"root_mean_squared_error": root_mean_squared_error})
models.append(mod)

In [ ]:
folds = 3 

In [ ]:
from tqdm import tqdm
i=0
res = np.zeros((test.shape[0]),dtype=np.float32)
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/step_size)))):
    for_prediction = get_keras_data(test.iloc[i:i+step_size], numericals, categoricals)
    res[i:min(i+step_size,test.shape[0])] = \
       np.expm1(models[0].predict(for_prediction, batch_size=1024)[:,0]) 
    i+=step_size

In [ ]:
len(res) 

In [ ]:
res[2260080] 

In [ ]:
np.argwhere(np.isnan(res))

In [ ]:
test.loc[[2260080]]

In [ ]:
submission = pd.read_csv(path + '/data-vol-1/sample_submission.csv')

In [ ]:
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0 

In [ ]:
submission.head(150)  

In [ ]:
leaked = feather.read_dataframe(path + '/data-vol-3/leaked_submission.feather')

In [ ]:
print(leaked.shape)

In [ ]:
leaked = leaked.dropna()
print(leaked.shape)

In [ ]:
leaked.head() 

In [ ]:
submission.set_index('row_id', inplace=True)

In [ ]:
#submission = submission.replace([np.inf, -np.inf], 0)

In [ ]:
predicted = np.log1p(submission.loc[leaked.index,'meter_reading'])
real = np.log1p(leaked['meter_reading'])
RMSLE = np.sqrt(np.mean(np.power(real-predicted,2)))
print(RMSLE.round(3))

In [ ]:
%%time
submission.loc[leaked.index,'meter_reading'] = leaked['meter_reading']
submission['meter_reading'] = submission['meter_reading'].round(4)
submission.head()

In [ ]:
submission.to_csv('nn_imputed_nn_3.csv',index = False)

In [ ]:
leaked.index[~pd.Series(leaked.index).isin(submission.index)]

In [ ]:
submission.head(100)